In [1]:
# ==========================================================
# BAGIAN 1: SETUP LINGKUNGAN
# ==========================================================
print(">>> [INFO] Meng-install semua library yang dibutuhkan...")

# Install semua paket yang dibutuhkan untuk aplikasi gabungan
!pip install -q -U \
    langchain \
    langchain-core \
    langchain-community \
    langchain-huggingface \
    langchain-text-splitters \
    langchain-groq \
    sentence-transformers \
    pypdf \
    faiss-cpu

# (Opsional) Hapus warning tokenizers
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(">>> [SUKSES] Semua library berhasil di-install/di-update!")
print(">>> [SANGAT PENTING!!!] Harap RESTART KERNEL SEKARANG sebelum melanjutkan!")
print("-" * 50)


>>> [INFO] Meng-install semua library yang dibutuhkan...
>>> [SUKSES] Semua library berhasil di-install/di-update!
>>> [SANGAT PENTING!!!] Harap RESTART KERNEL SEKARANG sebelum melanjutkan!
--------------------------------------------------


In [2]:
# --- Bagian 2: Load API Key & LLM (Versi VS Code) ---
import os
from dotenv import load_dotenv # Import library baru
from langchain_groq import ChatGroq
import os
import json
import re
from operator import itemgetter
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


print(">>> [INFO] Memuat GROQ_API_KEY...")

# 1. Coba load variabel dari file .env (jika ada)
load_dotenv()

# 2. Ambil API key dari environment variable
api_key = os.environ.get("GROQ_API_KEY")

# 3. Cek apakah key berhasil dimuat
if not api_key:
    # Kasih instruksi kalau key nggak ketemu
    raise EnvironmentError(
        "GROQ_API_KEY not found. "
        "Please ensure you have a .env file in the same directory with GROQ_API_KEY='YourKey', "
        "or that the environment variable is set."
    )
else:
    # Set ke os.environ jika belum (meski get() sudah cukup, kadang library lain butuh ini)
    os.environ["GROQ_API_KEY"] = api_key
    print(">>> [SUKSES] GROQ_API_KEY berhasil dimuat dari environment.")

# --- Inisialisasi LLM ---
# (Kode inisialisasi llm = ChatGroq(...) tetap sama, TIDAK PERLU DIUBAH)
llm = ChatGroq(
    model_name="openai/gpt-oss-20b", # atau model lain
    temperature=0.1
    # api_key=api_key # Bisa juga di-pass langsung jika library-nya support
)
print(f">>> [SUKSES] LLM Groq ({llm.model_name}) berhasil dimuat!")
print("-" * 50)

/home/philibra/anaconda3/envs/tutorial/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> [INFO] Memuat GROQ_API_KEY...
>>> [SUKSES] GROQ_API_KEY berhasil dimuat dari environment.
>>> [SUKSES] LLM Groq (openai/gpt-oss-20b) berhasil dimuat!
--------------------------------------------------


In [3]:
# ==========================================================
# BAGIAN 3: SETUP RAG (LOAD VECTOR STORES - VERSI VS CODE / LOKAL)
# ==========================================================
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os # Pastikan 'os' sudah di-import di awal skrip

# --- Konfigurasi Path LOKAL & Embedding ---
# GANTI path ini sesuai dengan NAMA FOLDER & LOKASI di proyek VS Code kamu.
# Contoh ini mengasumsikan folder index ada di folder yang sama dengan skrip .py ini.
# Jika kamu taruh di subfolder (misal 'data'), ubah jadi "data/faiss_index_task1", dll.
index_path_t1 = "/home/philibra/langchainTutorial/writingtask1ragdataset/faiss_index" # GANTI JIKA PERLU (nama folder index Task 1)
index_path_t2 = "/home/philibra/langchainTutorial/writingtask2ragdataset/faiss_index" # GANTI JIKA PERLU (nama folder index Task 2)

embedding_model_id = "BAAI/bge-large-en-v1.5" # HARUS SAMA dengan saat index dibuat

print(f">>> [INFO] Memuat Embedding Model: {embedding_model_id}...")
try:
    # Pastikan sentence-transformers terinstall: pip install sentence-transformers
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)
    print(">>> [SUKSES] Embedding Model dimuat.")
except Exception as e:
    raise RuntimeError(f"Gagal memuat embedding model: {e}")

# --- Load Vector Stores & Buat Retrievers ---
retriever_t1 = None
retriever_t2 = None

print("\n>>> [INFO] Memuat Vector Store LOKAL & Membuat Retriever...")
try:
    print(f">> Memuat Task 1 dari folder lokal: '{index_path_t1}'...")
    # Cek apakah folder path-nya ada
    if not os.path.exists(index_path_t1):
        raise FileNotFoundError(
            f"Folder Index Task 1 tidak ditemukan di '{index_path_t1}'. "
            "Pastikan folder sudah didownload dan path di kode sudah benar."
        )
    # Pastikan faiss-cpu terinstall: pip install faiss-cpu
    vectorstore_t1 = FAISS.load_local(index_path_t1, embeddings, allow_dangerous_deserialization=True)
    retriever_t1 = vectorstore_t1.as_retriever(search_kwargs={"k": 5}) # Sesuaikan 'k' atau tambahkan re-ranking jika perlu
    print(">> [SUKSES] Retriever Task 1 dibuat!")
except Exception as e:
    print(f"!!! [ERROR] Gagal memuat/membuat retriever Task 1: {e}.")
    # Berikan saran lebih spesifik jika mungkin error library
    if 'faiss' in str(e).lower():
        print("   Pastikan library 'faiss-cpu' sudah terinstall (`pip install faiss-cpu`).")

try:
    print(f">> Memuat Task 2 dari folder lokal: '{index_path_t2}'...")
    if not os.path.exists(index_path_t2):
        raise FileNotFoundError(
            f"Folder Index Task 2 tidak ditemukan di '{index_path_t2}'. "
            "Pastikan folder sudah didownload dan path di kode sudah benar."
        )
    vectorstore_t2 = FAISS.load_local(index_path_t2, embeddings, allow_dangerous_deserialization=True)
    retriever_t2 = vectorstore_t2.as_retriever(search_kwargs={"k": 5}) # Sesuaikan 'k' atau tambahkan re-ranking jika perlu
    print(">> [SUKSES] Retriever Task 2 dibuat!")
except Exception as e:
    print(f"!!! [ERROR] Gagal memuat/membuat retriever Task 2: {e}.")
    if 'faiss' in str(e).lower():
        print("   Pastikan library 'faiss-cpu' sudah terinstall (`pip install faiss-cpu`).")


# Berhenti jika salah satu retriever gagal dimuat
if not retriever_t1 or not retriever_t2:
    raise RuntimeError(
        "Gagal membuat salah satu atau kedua Retriever. "
        "Periksa pesan error di atas. Pastikan path folder index benar, "
        "library faiss-cpu terinstall, dan index dibuat dengan embedding model yang sama."
        )

print("\n>>> [SUKSES] Semua komponen RAG (Embeddings, Vector Stores LOKAL, Retrievers) siap.")
print("-" * 50)

>>> [INFO] Memuat Embedding Model: BAAI/bge-large-en-v1.5...
>>> [SUKSES] Embedding Model dimuat.

>>> [INFO] Memuat Vector Store LOKAL & Membuat Retriever...
>> Memuat Task 1 dari folder lokal: '/home/philibra/langchainTutorial/writingtask1ragdataset/faiss_index'...
>> [SUKSES] Retriever Task 1 dibuat!
>> Memuat Task 2 dari folder lokal: '/home/philibra/langchainTutorial/writingtask2ragdataset/faiss_index'...
>> [SUKSES] Retriever Task 2 dibuat!

>>> [SUKSES] Semua komponen RAG (Embeddings, Vector Stores LOKAL, Retrievers) siap.
--------------------------------------------------


In [4]:
# ==========================================================
# BAGIAN 4: DEFINISI SEMUA PROMPT & CHAIN
# ==========================================================

from langchain_core.prompts import PromptTemplate
# --- Definisi Semua Prompt Templates ---

# (PROMPT 1.1) EVALUASI TASK 1
template_evaluasi_task1 = """<|system|>
IELTS Examiner for Task 1 (Letters). Evaluate ONLY on CONTEXT.
VALIDATE TASK: IF Task 2 -> JSON error: {{"error": "SOAL DITOLAK. Ini Task 2. Model ini HANYA Task 1."}}.
IF Task 1: Evaluate TA (IF < 140 words, TA <= 5.0, comment must mention under-length), CC, LR, GRA based ONLY on CONTEXT. Comments: detailed, specific, quote examples. Output: SINGLE RAW JSON, EXACT format below.
EXAMPLE: {{ "Task Achievement": {{"band": 7.0, "comments": "..."}}, "Coherence & Cohesion": {{"band": 7.0, "comments": "..."}}, "Lexical Resource": {{"band": 6.0, "comments": "..."}}, "Grammatical Range & Accuracy": {{"band": 6.0, "comments": "..."}} }}<|end_system|>
<|user|>Evaluate. CONTEXT:{context} SOAL:{soal_prompt} ESSAY:{jawaban_essay}<|end_user|><|assistant|>"""
prompt_evaluasi_task1 = PromptTemplate.from_template(template_evaluasi_task1)

# (PROMPT 1.2) EVALUASI TASK 2
template_evaluasi_task2 = """<|system|>
IELTS Examiner for Task 2 (Essays). Evaluate ONLY on CONTEXT.
VALIDATE TASK: IF Task 1 -> JSON error: {{"error": "SOAL DITOLAK. Ini Task 1. Model ini HANYA Task 2."}}.
IF Task 2: Evaluate TR (IF < 220 words, TR <= 5.0, comment must mention under-length), CC, LR, GRA based ONLY on CONTEXT. Comments: detailed, specific, quote examples. Output: SINGLE RAW JSON, EXACT format below.
EXAMPLE: {{ "Task Response": {{"band": 7.0, "comments": "..."}}, "Coherence & Cohesion": {{"band": 7.0, "comments": "..."}}, "Lexical Resource": {{"band": 6.0, "comments": "..."}}, "Grammatical Range & Accuracy": {{"band": 6.0, "comments": "..."}} }}<|end_system|>
<|user|>Evaluate. CONTEXT:{context} SOAL:{soal_prompt} ESSAY:{jawaban_essay}<|end_user|><|assistant|>"""
prompt_evaluasi_task2 = PromptTemplate.from_template(template_evaluasi_task2)

# (PROMPT 2.1) SARAN TASK 1
template_saran_task1 = """<|system|>IELTS Coach. Give 3 actionable suggestions for the **letter** (ESSAY) based on EVALUATION JSON. Be specific, quote examples, focus on biggest problems. Respond ONLY with numbered list.<|end_system|><|user|>SOAL:{soal} ESSAY:{jawaban} EVALUATION:{evaluasi_json} Suggestions?<|end_user|><|assistant|>"""
prompt_saran_task1 = PromptTemplate.from_template(template_saran_task1)

# (PROMPT 2.2) SARAN TASK 2
template_saran_task2 = """<|system|>IELTS Coach. Give 3 actionable suggestions for the **essay** (ESSAY) based on EVALUATION JSON. Be specific, quote examples, focus on biggest problems. Respond ONLY with numbered list.<|end_system|><|user|>SOAL:{soal} ESSAY:{jawaban} EVALUATION:{evaluasi_json} Suggestions?<|end_user|><|assistant|>"""
prompt_saran_task2 = PromptTemplate.from_template(template_saran_task2)

# (PROMPT 3) PROOFREAD
template_proofread = """<|system|>Proofreader. Rewrite ESSAY. Mark errors: `~~salah~~ **benar**`. Unnecessary: `~~word~~`. Missing: `**word**`. Output ONLY corrected essay. Example: `I am ~~write~~ **writing**... The ~~equipments are~~ **equipment is** broken.`<|end_system|><|user|>Proofread: {jawaban}<|end_user|><|assistant|>"""
prompt_proofread = PromptTemplate.from_template(template_proofread)

# (PROMPT 4.1) REWRITE TASK 1
template_rewrite_task1 = """<|system|>IELTS Rewriter for Task 1. Rewrite ESSAY (letter) to TARGET BAND {target_band}. Use EVALUATION JSON. Improve TA, CC, LR, GRA. After essay, add '--- Why this achieves Band {target_band} ---' explaining improvements per criteria. Output ONLY rewritten essay & explanation.<|end_system|><|user|>SOAL:{soal} ORIGINAL ESSAY:{jawaban} INITIAL EVALUATION:{evaluasi_json} TARGET BAND:{target_band}. Rewrite.<|end_user|><|assistant|>"""
prompt_rewrite_task1 = PromptTemplate.from_template(template_rewrite_task1)

# (PROMPT 4.2) REWRITE TASK 2
template_rewrite_task2 = """<|system|>IELTS Rewriter for Task 2. Rewrite ESSAY to TARGET BAND {target_band}. Use EVALUATION JSON. Improve TR, CC, LR, GRA. After essay, add '--- Why this achieves Band {target_band} ---' explaining improvements per criteria. Output ONLY rewritten essay & explanation.<|end_system|><|user|>SOAL:{soal} ORIGINAL ESSAY:{jawaban} INITIAL EVALUATION:{evaluasi_json} TARGET BAND:{target_band}. Rewrite.<|end_user|><|assistant|>"""
prompt_rewrite_task2 = PromptTemplate.from_template(template_rewrite_task2)

# (PROMPT 5) CLASSIFIER
template_classifier = """<|system|>Classify SOAL: Task 1 (letter) or Task 2 (essay)? Hints T1:"letter", bullet points. T2:"agree/disagree", "discuss". Respond ONLY 'task_1' or 'task_2'.<|end_system|><|user|>SOAL:{soal} Classification:<|end_user|><|assistant|>"""
prompt_classifier = PromptTemplate.from_template(template_classifier)

print(">> [SUKSES] Semua prompt templates dibuat.")

# --- Definisi Semua Chains ---
# (Menggunakan llm, retriever_t1, retriever_t2 dari sel sebelumnya)

# Chain T1
rag_chain_t1 = ( RunnableParallel(context=itemgetter("jawaban") | retriever_t1, soal_prompt=itemgetter("soal"), jawaban_essay=itemgetter("jawaban")) | prompt_evaluasi_task1 | llm | StrOutputParser() )
chain_saran_t1 = prompt_saran_task1 | llm | StrOutputParser()
chain_rewrite_t1 = prompt_rewrite_task1 | llm | StrOutputParser()

# Chain T2
rag_chain_t2 = ( RunnableParallel(context=itemgetter("jawaban") | retriever_t2, soal_prompt=itemgetter("soal"), jawaban_essay=itemgetter("jawaban")) | prompt_evaluasi_task2 | llm | StrOutputParser() )
chain_saran_t2 = prompt_saran_task2 | llm | StrOutputParser()
chain_rewrite_t2 = prompt_rewrite_task2 | llm | StrOutputParser()

# Chain Umum
chain_proofread = prompt_proofread | llm | StrOutputParser()
chain_classifier = prompt_classifier | llm | StrOutputParser()

print(">> [SUKSES] Semua LangChain chains dibuat!")
print("-" * 50)

>> [SUKSES] Semua prompt templates dibuat.
>> [SUKSES] Semua LangChain chains dibuat!
--------------------------------------------------


In [5]:
# ==========================================================
# BAGIAN 5: DEFINISI FUNGSI HELPER
# ==========================================================

def hitung_skor_keseluruhan(skor_dict):
    """Menghitung skor keseluruhan."""
    try:
        tr_key = next(key for key in skor_dict if "Task" in key)
        tr = float(skor_dict[tr_key]["band"])
        cc = float(skor_dict["Coherence & Cohesion"]["band"])
        lr = float(skor_dict["Lexical Resource"]["band"])
        gra = float(skor_dict["Grammatical Range & Accuracy"]["band"])
        rata_rata = (tr + cc + lr + gra) / 4.0
        sisa = rata_rata - int(rata_rata)
        if sisa >= 0.75: return int(rata_rata) + 1.0
        elif sisa >= 0.25: return int(rata_rata) + 0.5
        else: return float(int(rata_rata))
    except Exception as e:
        print(f"[Debug] Gagal hitung skor awal: {e}")
        return "N/A"

def tampilkan_laporan(task_type, evaluasi_string, word_count, saran_perbaikan, saran_proofread):
    """Menampilkan laporan awal (disesuaikan per task)."""
    print("\n" + "📝" * 15 + f" HASIL PEMERIKSAAN AWAL ({task_type.upper()}) " + "📝" * 15)
    data, skor_akhir_awal = None, "N/A"
    min_words = 150 if task_type == 'task_1' else 250

    try:
        match = re.search(r'\{.*\}', evaluasi_string, re.DOTALL)
        json_str = match.group(0) if match else evaluasi_string
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("\n" + "="*25 + " EVALUASI GAGAL (OUTPUT BUKAN JSON) " + "="*25); print(f"Output mentah:\n{evaluasi_string}"); print("-" * 70)
        return None, "N/A"

    print(f"\n 📝 JUMLAH KATA: {word_count} kata")
    if word_count < min_words: print(f"   ⚠️ PERINGATAN: Di bawah {min_words} kata (syarat minimum {task_type.upper()}).")

    print("\n" + "✍️" * 15 + " KOREKSI DETAIL (PROOFREAD) " + "✍️" * 15); print(saran_proofread); print("-" * 50)

    if "error" in data:
        print("\n" + "="*25 + " EVALUASI DIBATALKAN " + "="*25); print(f"\n>>> {data['error']} <<<"); print("\n" + "=" * 70)
        return data, "N/A"

    print("\n" + "📊" * 15 + " HASIL EVALUASI SKOR " + "📊" * 15)
    skor_akhir_awal = hitung_skor_keseluruhan(data)
    print(f"\n band_keseluruhan: {skor_akhir_awal}"); print("-" * 50)

    kriteria_keys = list(data.keys())
    emojis = {"Task": "🎯", "Coherence": "🔗", "Lexical": "📚", "Grammatical": "🖋️"}

    for key in kriteria_keys:
        emoji = "📊"
        for k_emoji, v_emoji in emojis.items():
            if k_emoji in key: emoji = v_emoji; break

        if isinstance(data[key], dict):
            detail = data[key]
            band = detail.get("band", "N/A")
            comments = detail.get("comments", "Tidak ada komentar.")
            print(f"\n{emoji} {key.upper()}"); print(f"  Skor: {band}"); print(f"  Komentar: {comments}")
        else:
             print(f"\n{emoji} {key.upper()} (Format Skor Tidak Dikenal)")

    print("\n" + "💡" * 15 + " SARAN PERBAIKAN UMUM " + "💡" * 15); print(saran_perbaikan)
    print("\n" + "=" * 70)
    return data, skor_akhir_awal

print(">>> [SUKSES] Fungsi helper (hitung skor, tampilkan laporan) siap.")
print("-" * 50)

>>> [SUKSES] Fungsi helper (hitung skor, tampilkan laporan) siap.
--------------------------------------------------


In [ ]:
# ==========================================================
# BAGIAN 6: LOOP INTERAKTIF GABUNGAN
# ==========================================================
print("\n" + "=" * 20 + " AI IELTS EXAMINER (TASK 1 & 2) SIAP " + "=" * 20)

while True:
    print("\n" + "="*50) # Pemisah antar sesi
    soal_ielts = input(">>> Masukkan SOAL IELTS (Task 1 atau 2) (atau 'exit'): ")
    if soal_ielts.lower() == 'exit': print("\nTerima kasih!"); break

    print("\n>>> Masukkan JAWABAN/ESAI Anda (Ketik 'END' di baris baru utk selesai):")
    jawaban_lines = []
    while True:
        try:
            line = input()
            if line.strip().upper() == 'END': break
            if line.strip().lower() == 'exit': jawaban_lines.append('exit'); break
            jawaban_lines.append(line)
        except EOFError: break
    if 'exit' in [l.strip().lower() for l in jawaban_lines]: print("\nTerima kasih!"); break

    esai_kandidat = "\n".join(jawaban_lines)
    if not esai_kandidat.strip() or not soal_ielts.strip(): print(">>> [ERROR] Soal/Jawaban kosong."); continue

    word_count = len(esai_kandidat.split())

    try:
        # Tahap 1: Klasifikasi Task
        print(f"\n>>> [INFO] Mengklasifikasi jenis task...")
        task_type = chain_classifier.invoke({"soal": soal_ielts}).strip().lower().replace("'", "") # Bersihkan output classifier
        print(f">>> [INFO] Terdeteksi sebagai: {task_type}")

        if task_type not in ['task_1', 'task_2']:
            print(f">>> [ERROR] Tidak bisa menentukan jenis task ('{task_type}'). Coba perjelas soal.")
            continue

        # Pilih chain yang sesuai
        if task_type == 'task_1':
            eval_chain, saran_chain, rewrite_chain = rag_chain_t1, chain_saran_t1, chain_rewrite_t1
        else: # task_2
            eval_chain, saran_chain, rewrite_chain = rag_chain_t2, chain_saran_t2, chain_rewrite_t2

        # Tahap 2: Evaluasi JSON
        print(f">>> [INFO] Model ({llm.model_name}) berpikir (Tahap 1: Evaluasi JSON)...")
        input_eval = {"soal": soal_ielts, "jawaban": esai_kandidat}
        hasil_evaluasi_string = eval_chain.invoke(input_eval)

        # Tahap 3: Koreksi Detail
        print(f">>> [INFO] Model ({llm.model_name}) berpikir (Tahap 2: Koreksi Detail)...")
        saran_proofread = chain_proofread.invoke({"jawaban": esai_kandidat})

        # Tahap 4: Parse & Cek Error
        data_evaluasi, skor_awal_float, evaluasi_valid = None, None, False
        skor_awal_display = "N/A"
        try:
            match = re.search(r'\{.*\}', hasil_evaluasi_string, re.DOTALL)
            json_str = match.group(0) if match else hasil_evaluasi_string
            data_evaluasi = json.loads(json_str)
            if "error" not in data_evaluasi: # Cek apakah ada error dari prompt validasi
                skor_awal_display = hitung_skor_keseluruhan(data_evaluasi)
                if skor_awal_display != "N/A":
                    skor_awal_float = float(skor_awal_display)
                    evaluasi_valid = True
        except Exception as parse_e:
             print(f"[Debug] Gagal parse JSON evaluasi awal: {parse_e}")
             evaluasi_valid = False

        saran_perbaikan = "(Dilewati karena evaluasi awal gagal/tidak valid)"
        if evaluasi_valid:
            # Tahap 5: Minta Saran Umum
            print(f">>> [INFO] Model ({llm.model_name}) berpikir (Tahap 3: Minta Saran)...")
            input_saran = {"soal": soal_ielts, "jawaban": esai_kandidat, "evaluasi_json": hasil_evaluasi_string}
            saran_perbaikan = saran_chain.invoke(input_saran)

        # Tahap 6: Tampilkan Laporan Awal
        data_json, skor_awal_display_final = tampilkan_laporan(
            task_type, hasil_evaluasi_string, word_count,
            saran_perbaikan, saran_proofread
        )

        # Tahap 7: Tanya Revisi
        if evaluasi_valid and skor_awal_float is not None:
            while True:
                resp = input(f"\n>>> Skor awal: {skor_awal_display_final}. Revisi ke Band lebih tinggi? (y/n): ").lower()
                if resp == 'n': break
                elif resp == 'y':
                    while True:
                        try:
                            t_band_str = input(f">>> Target Band (> {skor_awal_display_final}, maks 9.0): ")
                            t_band = float(t_band_str)
                            if t_band > skor_awal_float and t_band <= 9.0:
                                print(f"\n>>> [INFO] Model ({llm.model_name}) berpikir (Tahap 4: Revisi ke Band {t_band})...")
                                input_rewrite = {
                                    "soal": soal_ielts, "jawaban": esai_kandidat,
                                    "evaluasi_json": hasil_evaluasi_string, "target_band": t_band
                                }
                                hasil_rewrite = rewrite_chain.invoke(input_rewrite)
                                print("\n" + "✨" * 15 + f" VERSI REVISI (TARGET BAND {t_band}) " + "✨" * 15)
                                print(hasil_rewrite); print("\n" + "=" * 70)
                                break
                            else: print(f"   [ERROR] Target > {skor_awal_display_final} & <= 9.0.")
                        except ValueError: print("   [ERROR] Masukkan angka valid (misal: 7.0, 7.5).")
                    break
                else: print("   [ERROR] Jawab 'y' atau 'n'.")

    except Exception as e:
        print(f"\n>>> [ERROR] Terjadi masalah: {e}") # <-- Sudah print 'e' di kode terakhir, bagus!
        import traceback # Uncomment jika perlu detail
        print(traceback.format_exc()) # Uncomment jika perlu detail
        print(">>> Pastikan API Key valid & ada kuota. Cek juga path dataset RAG.")


==================== AI IELTS EXAMINER (TASK 1 & 2) SIAP ====================




>>> Masukkan JAWABAN/ESAI Anda (Ketik 'END' di baris baru utk selesai):
>>> [ERROR] Soal/Jawaban kosong.


>>> Masukkan JAWABAN/ESAI Anda (Ketik 'END' di baris baru utk selesai):

>>> [INFO] Mengklasifikasi jenis task...
>>> [INFO] Terdeteksi sebagai: task_1
>>> [INFO] Model (openai/gpt-oss-20b) berpikir (Tahap 1: Evaluasi JSON)...
>>> [INFO] Model (openai/gpt-oss-20b) berpikir (Tahap 2: Koreksi Detail)...
>>> [INFO] Model (openai/gpt-oss-20b) berpikir (Tahap 3: Minta Saran)...

📝📝📝📝📝📝📝📝📝📝📝📝📝📝📝 HASIL PEMERIKSAAN AWAL (TASK_1) 📝📝📝📝📝📝📝📝📝📝📝📝📝📝📝

 📝 JUMLAH KATA: 161 kata

✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️ KOREKSI DETAIL (PROOFREAD) ✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️✍️
Dear Tom,

I hope you are doing well. I just wanted to tell you that I have moved to Jakarta because I got a new job here. The city is very busy and full of traffic, but I like it because there are many restaurants and shopping malls around my apartment.

The reason I moved here ~~is because~~ **is that** my company opened a new offi